In [ ]:
# @title Importing Libraries
import torch
!pip install "unsloth[colab] @ git+https://github.com/unslothai/unsloth.git"
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
from transformers.utils import logging
from transformers import TextIteratorStreamer
from threading import Thread
import textwrap
import random

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-l1rhfh0v/unsloth_efe480b361824fbf88dcc31777b707af
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-l1rhfh0v/unsloth_efe480b361824fbf88dcc31777b707af
  Resolved https://github.com/unslothai/unsloth.git to commit 03f2c646d470760d4f4834dc3e3b1f959265a0ba
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached https://download.pytorch.org/whl/cu121/xformers-0.0.22.post7-cp310-cp310-manylinux2014_x86_64.whl (211.8 MB)


In [ ]:
# @title Loading Pre-trained Model (Mistral) and its Tokenizer
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)


In [ ]:
# @title Data Preprocessing
dataset = load_dataset("roneneldan/TinyStories", split = "train[:30]")

EOS_TOKEN = tokenizer.eos_token
def formatting_func(example):
  return example["text"] + EOS_TOKEN

In [ ]:
# @title Retrieving Computer Capabilities
major_version, minor_version = torch.cuda.get_device_capability()

In [ ]:
# @title Adjusting Model's Hyperparameters

model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128

    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",

                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = True,
    random_state = 3407,
    max_seq_length = max_seq_length,
)

In [ ]:
# @title Model Finetuning

logging.set_verbosity_info()

trainer = SFTTrainer(
    model = model,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    packing = True,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_ratio = 0.1,
        num_train_epochs = 1,
        learning_rate = 2e-5,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.1,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)
trainer_stats = trainer.train()

In [ ]:
# @ title Prompt Engineering
# alpaca_prompt="""
# [INST] You are trained on different stories. You will run a game. You will generate a scenerio from the stories and 4 options from the story.
# [/INST]

# """

In [ ]:
# @title Generating Text

text_streamer = TextIteratorStreamer(tokenizer)
max_print_width = 100

# inputs = tokenizer(
#     ["Once upon a time there was a king "] * 1, return_tensors="pt"
# ).to("cuda")

# generation_kwargs = dict(
#     inputs,
#     streamer=text_streamer,
#     max_new_tokens=256,
#     use_cache=True,
# )
# thread = Thread(target=model.generate, kwargs=generation_kwargs)
# thread.start()

# length = 0
# for j, new_text in enumerate(text_streamer):
#     if j==0:
#         wrapped_text = textwrap.wrap(new_text, width = max_print_width)
#         length = len(wrapped_text [-1])
#         wrapped_text = "\n" . join(wrapped_text)
#         print(wrapped_text, end = "")
#     else:
#         length += len(new_text)
#         if length >= max_print_width:
#             length = 0
#             print()
#         print(new_text, end = "")
#     pass
# pass
import random

# Define the prompt and options
prompt = "Once upon a time there was a king"
options = ["who ruled over a prosperous kingdom.",
           "who was known for his wisdom and fairness.",
           "who had a magical crown that granted wishes.",
           "who had a loyal advisor named Merlin."]

# Shuffle the options and identify the correct one
correct_option = options.index("who ruled over a prosperous kingdom.")
random.shuffle(options)

# Construct the prompt with options
prompt_with_options = f"{prompt} {options[0]}, {options[1]}, {options[2]}, or {options[3]}?"

# Tokenize the prompt with options
inputs = tokenizer(prompt_with_options, return_tensors="pt").to("cuda")

# Generate the completion of the story
generation_kwargs = dict(
    inputs,
    streamer=text_streamer,
    max_new_tokens=256,
    use_cache=True,
)
thread = Thread(target=model.generate, kwargs=generation_kwargs)
thread.start()

# Print the generated text and options
length = 0
for j, new_text in enumerate(text_streamer):
    if j == 0:
        wrapped_text = textwrap.wrap(new_text, width=max_print_width)
        length = len(wrapped_text[-1])
        wrapped_text = "\n".join(wrapped_text)
        print(wrapped_text, end="")
    else:
        length += len(new_text)
        if length >= max_print_width:
            length = 0
            print()
        print(new_text, end="")

# Print the shuffled options
print("\n\nOptions:")
for i, option in enumerate(options):
    print(f"{i+1}. {option}")

# Print the correct option
print(f"\nThe correct completion is option {correct_option+1}.")
text_streamer